## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### <font color="red"> Data</font>

In [1]:
#run 'Pkg.clone("https://github.com/QTL-rocks/QTLDatasets.jl.git")' to get test datasets
using DataFrames
using QTLDatasets
using JWAS.ST

In [3]:
phenofile = QTLDatasets.dataset("testMME","simple.txt")
genofile  = QTLDatasets.dataset("testMME","genotype.txt")
pedfile   = QTLDatasets.dataset("testMME","pedigree.txt");

In [4]:
phenotype = readtable(phenofile,separator = ',',header=true)

,Animal,Age,y
1,S1,1,-0.92
2,D1,2,-1.05
3,O1,3,-0.92
4,O3,2,1.2


In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1,0,1,1,1
D1,2,0,2,2,1
O1,1,2,0,1,0
O3,0,0,2,1,1


### <font color="red"> Run module</font>

In [6]:
varRes = 1.0
varGen = 2.5
markerProp = 0.8
MVarGen= varGen*markerProp
PVarGen= varGen*(1-markerProp);

In [7]:
ped = get_pedigree(pedfile);

In [8]:
varRes=1.0
mme = build_model("y = intercept + Age + Animal",varRes)
set_covariate(mme,"Age")
set_random(mme,"Animal",ped,PVarGen)

### <font color="red"> work with markers</font>

In [9]:
add_markers(mme,genofile,MVarGen,separator=',');

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMME/genotype.txt is ,  .


### <font color="red"> run MCMC</font>

In [10]:
output=runMCMC(mme,phenotype,chain_length=50000,
                         printout_frequency=5000,methods="BayesC",Pi=0.8,
                         output_marker_effects_frequency=1000)

at sample: 5000 with meanVare: 1.180020941204773
at sample: 10000 with meanVare: 1.1290584794704177
at sample: 15000 with meanVare: 1.1564789627265653
at sample: 20000 with meanVare: 1.1568634335108294
at sample: 25000 with meanVare: 1.1473013496017428
at sample: 30000 with meanVare: 1.147098579656586
at sample: 35000 with meanVare: 1.1517152527580927
at sample: 40000 with meanVare: 1.1602999832985028
at sample: 45000 with meanVare: 1.1529707294334304
at sample: 50000 with meanVare: 1.1539114497497545


Dict{Any,Any} with 4 entries:
  "Posterior Mean of Loca… => 7x2 Array{Any,2}:…
  "MCMC samples for resid… => [0.749379,0.565092,1.34446,0.782293,1.39658,0.483…
  "MCMC samples for genet… => 50000x1 Array{Float64,2}:…
  "Posterior Mean of Mark… => [-0.28637301435446094,-0.14759747418073482,0.0904…

In [11]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior Mean of Location Parameters"
  "MCMC samples for residual variance"
  "MCMC samples for genetic var-cov parameters"
  "Posterior Mean of Marker Effects"

In [12]:
output["Posterior Mean of Location Parameters"]

7x2 Array{Any,2}:
 "intercept: intercept"  -1.05764    
 "Age: Age"               0.307154   
 "Animal: S1"             0.00808428 
 "Animal: D1"            -0.00753495 
 "Animal: O1"            -0.0316165  
 "Animal: O3"             0.0958422  
 "Animal: O2"            -0.000453417

In [13]:
output["Posterior Mean of Marker Effects"]

5-element Array{Float64,1}:
 -0.286373 
 -0.147597 
  0.0904282
 -0.116729 
  0.154358 

In [14]:
output[ "MCMC samples for genetic var-cov parameters"]

50000x1 Array{Float64,2}:
 0.772699
 0.491373
 0.24032 
 0.356933
 0.214215
 0.159621
 0.235844
 0.210135
 0.475849
 0.261517
 0.22853 
 0.451648
 0.249645
 ⋮       
 0.157818
 0.134052
 0.227978
 0.223799
 0.312398
 0.175965
 0.204944
 0.181598
 0.223808
 0.386585
 0.16054 
 0.300723

In [15]:
output["MCMC samples for residual variance"]

50000-element Array{Float64,1}:
 0.749379
 0.565092
 1.34446 
 0.782293
 1.39658 
 0.483855
 1.19215 
 3.07654 
 1.11291 
 2.148   
 0.475404
 1.21807 
 0.572408
 ⋮       
 1.08261 
 0.673619
 1.2749  
 1.528   
 1.17411 
 0.797999
 0.640783
 1.05866 
 1.67794 
 0.301304
 0.569661
 0.571884